# Data Cleaning
Initial data cleaning and preprocessing for the project. Takes in data from `training_data.csv`, which was output from a SQL query from the CEIP database. This notebook is used to create the preprocessing pipeline in `pipeline.ipynb`. This used to clean the data for the exploratory data analysis and machine learning notebooks. It outputs a cleaned dataset to `CEIP_csv/cleaned.csv`.

#### Setup

In [6]:
# CORE
import pandas as pd
import os
import json
import numpy as np  # Numpy for numerical computations and array operations
import pandas as pd  # Pandas for data manipulation and analysis

# MACHINE LEARNING & STATISTICS 
import scipy.stats as stats  # SciPy for scientific computing and technical computing, including statistics
import sklearn as sk # Scikit-learn for machine learning and predictive modeling

# VISUALIZATION
import matplotlib.pyplot as plt  # Matplotlib for creating static, animated, and interactive visualizations
import seaborn as sns  # Seaborn for statistical data visualization built on top of Matplotlib
import plotly.express as px  # Plotly Express for creating interactive plots and charts
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
# filepaths of all of the csv data files to be analyzed

# autonest_csv = "CEIP_csv/AutoNest.csv"
# autonest_strategy_csv = "CEIP_csv/AutoNestStrategy.csv"
# material_csv = "CEIP_csv/Material.csv"
# nest_csv = "CEIP_csv/Nest.csv"
# part_csv = "CEIP_csv/Part.csv"
# performance_csv = "CEIP_csv/Performance.csv"
training_csv = "../CEIP_csv/training_data.csv"

# read in all of these csv files as pandas dataframes

# autonest_df = pd.read_csv(autonest_csv)
# autonest_strategy_df = pd.read_csv(autonest_strategy_csv)
# material_df = pd.read_csv(material_csv)
# nest_df = pd.read_csv(nest_csv)
# part_df = pd.read_csv(part_csv)
# performance_df = pd.read_csv(performance_csv)
training_df = pd.read_csv(training_csv) # takes about 1 min to read 

## Data Cleaning

#### Data Previewing

In [8]:
# count the number of null values 
# training_df.isnull().sum() # --> RESULT: no null values in any column  

# list of columns
# column_list = list(training_df.columns)
column_list = ['ixJobSummary', 'ixNest', 'ixPart', 'dPartTrueArea', 'cRequired', 
               'cNested', 'ixMaterial', 'fExtShape', 'dExtArea', 'dExtBoundaryDist', 
               'dExtContainedDist', 'dLgIntArea', 'dLgIntBoundaryDist', 'dLgIntContainedDist', 
               'dLgExtConArea', 'dLgExtConBoundaryDist', 'dLgExtConContainedDist', 'cTimesCut', 
               'dNestingTime', 'fStrategies', 'dSheetLength', 'dSheetWidth', 'dSheetArea', 'dLengthUsed', 
               'dWidthUsed', 'dPartArea', 'calcUtil', 'ixAutoNestStrategy', 'fAllPartsNested']

# count the values for the ixMaterial column in the training_df 
# training_df.ixMaterial.value_counts()

# plot the distribution of the ixMaterial column in the training_df
# only plot the 10 most common values
# training_df.ixMaterial.value_counts().nlargest(10).plot(kind='bar', figsize=(10,5))

# count the number of unique values
# training_df.nunique(axis=0)

# ixJobSummary               224892
# ixNest                     224892
# ixPart                    4200357
# dPartTrueArea              984974
# cRequired                    1534
# cNested                      1654
# ixMaterial                   7316
# fExtShape                      52
# dExtArea                   825580
# dExtBoundaryDist           159907
# dExtContainedDist          526206
# dLgIntArea                  74924
# dLgIntBoundaryDist          35721
# dLgIntContainedDist         83215
# dLgExtConArea               71156
# dLgExtConBoundaryDist       24334
# dLgExtConContainedDist      68328
# cTimesCut                     176
# dNestingTime                23997
# fStrategies                   292
# dSheetLength                 2917
# dSheetWidth                  1937
# dSheetArea                  13217
# dLengthUsed                136950
# dWidthUsed                 112688
# dPartArea                  168268
# calcUtil                   172012
# ixAutoNestStrategy             13
# fAllPartsNested                 2
# dtype: int64

In [22]:
# above shows that there are only 4.2 million unique values for ixPart 
# this indicates that there are a lot of rows that are duplicated? 

# remove duplicates 
training_df = training_df.drop_duplicates()
# can also drop duplicates from only specific rows 
# df.drop_duplicates(subset=df.columns[2:], keep=False, inplace=True)
training_df.shape # --> (5,762,622 rows, 29 columns)

KeyboardInterrupt: 

#### Re-encoding Variables
* One-hot encode all variables that are numerical but represent categories
* Replace ixMaterial and ixAutoNestStrategy with their appropriate values from the JSON file 
* Limit to only Materials that are steel

In [10]:
# replace ixMaterial column with the values from the MaterialTypes.json file 
# read in the json file as a pandas dataframe
material_types_df = pd.read_json('../CEIP_csv/MaterialTypes.json')

# create a dictionary of the material types and their corresponding values 
material_dict = material_types_df.to_dict()

# We need to make a new dictionary where the keys are the same but the values are the sNames
sName_dict = {k: v['sName'] for k, v in material_dict.items()}

# Now we use this dictionary to replace the ixMaterial values in training_df
training_df['ixMaterial'] = training_df['ixMaterial'].map(sName_dict)

# Rename the column
training_df = training_df.rename(columns={'ixMaterial': 'Material'})

In [11]:
# Count the different material types
material_counts = training_df['Material'].value_counts()
# results: 4,072,515 rows are MS, the next most is SS with 191,961 

# count the number of unique materials
nunique_materials = training_df.Material.nunique()
print(f'There are {nunique_materials} unique materials in the dataset.')

# Convert the Series to a DataFrame
material_counts_df = material_counts.reset_index()

# Rename the columns for clarity
material_counts_df.columns = ['Material', 'Count']

# Sort the DataFrame by the 'Count' column in descending order and take the top 30 rows
top_material_counts_df = material_counts_df.sort_values('Count', ascending=False).head(100)

# Create a bar chart for the top 30 materials
fig = px.bar(top_material_counts_df, x='Material', y='Count', title='Distribution of Top 100 Material Types')
fig.show()

There are 2117 unique materials in the dataset.


In [23]:
# remove all rows that do not have mild steel (ms) as the Material

# drop all the NA values from Material, Keep only the rows where Material contains 'ms'
training_df = training_df.dropna(subset=['Material'])
training_df = training_df[training_df['Material'] == 'ms']

# Drop the Material column from the dataframe - don't need it anymore 
encoded_df = training_df.drop(columns=['Material'])

In [25]:
# Load the dictionary from the JSON file
with open('../CEIP_csv/AutoNestStrategy.json', 'r') as f:
    autoneststrategy_dict = json.load(f)
    
#  Convert ixAutoNestStrategy to string
encoded_df['ixAutoNestStrategy'] = encoded_df['ixAutoNestStrategy'].astype(str)

# Replace the ixAutoNestStrategy values in the DataFrame
encoded_df['ixAutoNestStrategy'] = encoded_df['ixAutoNestStrategy'].map(autoneststrategy_dict)

# save this column for comparison for later use before one-hot encoding 
comparison_df = pd.DataFrame()
comparison_df['ixAutoNestStrategy'] = encoded_df['ixAutoNestStrategy']

# Count the different strategies
strategy_counts = encoded_df['ixAutoNestStrategy'].value_counts()

# Create a pie chart directly from the Series
fig = px.pie(strategy_counts, values=strategy_counts.values, names=strategy_counts.index, 
             title='Distribution of Strategies Used in AutoNest')
fig.show()

In [26]:
# one hot encoding for the AutoNestStrategy to convert categorical from numeric 

# Perform one-hot encoding for the ixAutoNestStrategy column
ix_auto_nest_encoded = pd.get_dummies(encoded_df['ixAutoNestStrategy'], prefix='AutoStrat')

# Concatenate the new columns to the original DataFrame
encoded_df = pd.concat([encoded_df, ix_auto_nest_encoded], axis=1)

# Optionally, you can drop the original ixAutoNestStrategy column
encoded_df = encoded_df.drop('ixAutoNestStrategy', axis=1)

encoded_df.head()

,ixJobSummary,ixNest,ixPart,dPartTrueArea,cRequired,cNested,fExtShape,dExtArea,dExtBoundaryDist,dExtContainedDist,...,AutoStrat_Strategy_1,AutoStrat_Strategy_10,AutoStrat_Strategy_2,AutoStrat_Strategy_3,AutoStrat_Strategy_4,AutoStrat_Strategy_5,AutoStrat_Strategy_6,AutoStrat_Strategy_7,AutoStrat_Strategy_8,AutoStrat_Strategy_9
0,304409,746165,2377314,151.1253,5,5,0,151.1253,4.7891,15.8795,...,0,0,0,0,1,0,0,0,0,0
1,304409,746165,2377315,49.5807,10,10,0,49.5807,3.1157,9.9701,...,0,0,0,0,1,0,0,0,0,0
3,304409,746165,2377316,222.0186,5,5,0,222.0186,4.7891,24.3234,...,0,0,0,0,1,0,0,0,0,0
5,304409,746165,2377317,2.5008,120,120,0,2.5008,0.6184,2.7209,...,0,0,0,0,1,0,0,0,0,0
7,304409,746165,2377318,11.5773,55,55,0,11.5773,0.9060,5.9537,...,0,0,0,0,1,0,0,0,0,0


In [27]:
# !! COMMENTED OUT: keep this as a bitmask for the ML models

# re-encode the fStrategies column 
# Define a dictionary with the bitmask values for each strategy
strategies_dict = {
    'Strategy1': 0x00000001,
    'Strategy2': 0x00000002,
    'Strategy3': 0x00000004,
    'Strategy4': 0x00000008,
    'Strategy5': 0x00000010,
    'Strategy6': 0x00000020,
    'Strategy7': 0x00000040,
    'Strategy8': 0x00000080,
    'Strategy9': 0x00000100,
    'Strategy10': 0x00000200,
    'BlockNesting': 0x00000400,
    'BlockOptimization': 0x00000800,
    'IntelliNest': 0x00001000,
    'ICProfileNesting': 0x00002000,
    'ICPatternFill': 0x00004000,
    'ManualNesting': 0x80000000
}

# convert fStrategy bitMasks to strings 
def get_strategy_name(bitmask, strategies_dict):
    for strategy, bitmask_value in strategies_dict.items():
        if bitmask & bitmask_value:
            return strategy
    return None

encoded_df['fStrategy'] = encoded_df['fStrategies'].apply(lambda x: get_strategy_name(x, strategies_dict))

# Drop the old fStrategies column
encoded_df = encoded_df.drop('fStrategies', axis=1)

# Rename the new fStrategy column to fStrategies
encoded_df = encoded_df.rename(columns={'fStrategy': 'fStrategies'})

# get the value counts of fStrategy types 
fstrat_value_counts = encoded_df['fStrategies'].value_counts()
print(fstrat_value_counts)

Strategy4            987912
ICProfileNesting     697235
ICPatternFill        391337
IntelliNest          313332
ManualNesting        279737
Strategy1            276538
Strategy10           272571
Strategy5             99022
Strategy3             76083
Strategy9             60085
BlockNesting          58917
Strategy8             46291
BlockOptimization     41814
Strategy2             41495
Strategy7             40244
Strategy6             37818
Name: fStrategies, dtype: int64


In [28]:
# graph the results with plotly 
# limit to just the top 10 most common strategies
top_fstrat_counts_df = fstrat_value_counts.head(10)
fig = go.Figure(go.Pie(labels=top_fstrat_counts_df.index, values=top_fstrat_counts_df.values, textinfo='label+percent'))
fig.update_layout(title='Distribution of the Top 10 fStrategies Used in Nests')
fig.show()

In [43]:
# count the number of NAs or Nones in fStrategies column 
na_count = encoded_df['fStrategies'].isna().sum()
print(f"Number of NAs or Nones in fStrategies column: {na_count}")

# fill in the missing values in fStrategies column with 'None'
encoded_df['fStrategies'] = encoded_df['fStrategies'].fillna('None')

Number of NAs or Nones in fStrategies column: 352084


In [29]:
# COMPARING fStrategies and ixAutoNestStrategy
# compare the value of fStrategies column with the values of ixAutoNestStrategy from the training_df
comparison_df['fStrategies'] = encoded_df['fStrategies']

# Create a new column to check if fStrategies and ixAutoNestStrategy are the same
comparison_df['is_same'] = comparison_df.apply(lambda row: row['fStrategies'] == row['ixAutoNestStrategy'], axis=1)

comparison_df
# Calculate the percentage of rows where fStrategies and ixAutoNestStrategy are the same
same_percentage = (comparison_df['is_same'].sum() / len(comparison_df)) * 100
print(f"Percentage of rows where fStrategies and ixAutoNestStrategy are the same: {same_percentage}%")

Number of NAs or Nones in fStrategies column: 352084
Percentage of rows where fStrategies and ixAutoNestStrategy are the same: 0.0%


In [46]:
# one-hot encoding of fStrategies
# one hot encoding for the fStrategies to convert categorical from numeric 
# Perform one-hot encoding for the ixAutoNestStrategy column
ix_auto_nest_encoded = pd.get_dummies(encoded_df['fStrategies'], prefix='fStrat')

# Concatenate the new columns to the original DataFrame
encoded_df = pd.concat([encoded_df, ix_auto_nest_encoded], axis=1)

# Optionally, you can drop the original ixAutoNestStrategy column
encoded_df = encoded_df.drop('fStrategies', axis=1)
# ?? Mark - what does it mean when it has a strategy in fStrategies AND one in ixAutoNestStrategy? 
# ?? Can a given Job use both manual nesting (chosen strategies in fStrategies) AND IntelliNest? 

,ixJobSummary,ixNest,ixPart,dPartTrueArea,cRequired,cNested,fExtShape,dExtArea,dExtBoundaryDist,dExtContainedDist,...,fStrat_Strategy1,fStrat_Strategy10,fStrat_Strategy2,fStrat_Strategy3,fStrat_Strategy4,fStrat_Strategy5,fStrat_Strategy6,fStrat_Strategy7,fStrat_Strategy8,fStrat_Strategy9
0,304409,746165,2377314,151.1253,5,5,0,151.1253,4.7891,15.8795,...,0,0,0,0,0,0,0,0,0,0
1,304409,746165,2377315,49.5807,10,10,0,49.5807,3.1157,9.9701,...,0,0,0,0,0,0,0,0,0,0
3,304409,746165,2377316,222.0186,5,5,0,222.0186,4.7891,24.3234,...,0,0,0,0,0,0,0,0,0,0
5,304409,746165,2377317,2.5008,120,120,0,2.5008,0.6184,2.7209,...,0,0,0,0,0,0,0,0,0,0
7,304409,746165,2377318,11.5773,55,55,0,11.5773,0.9060,5.9537,...,0,0,0,0,0,0,0,0,0,0


In [49]:
encoded_df.head()

,ixJobSummary,ixNest,ixPart,dPartTrueArea,cRequired,cNested,fExtShape,dExtArea,dExtBoundaryDist,dExtContainedDist,...,fStrat_Strategy1,fStrat_Strategy10,fStrat_Strategy2,fStrat_Strategy3,fStrat_Strategy4,fStrat_Strategy5,fStrat_Strategy6,fStrat_Strategy7,fStrat_Strategy8,fStrat_Strategy9
0,304409,746165,2377314,151.1253,5,5,0,151.1253,4.7891,15.8795,...,0,0,0,0,0,0,0,0,0,0
1,304409,746165,2377315,49.5807,10,10,0,49.5807,3.1157,9.9701,...,0,0,0,0,0,0,0,0,0,0
3,304409,746165,2377316,222.0186,5,5,0,222.0186,4.7891,24.3234,...,0,0,0,0,0,0,0,0,0,0
5,304409,746165,2377317,2.5008,120,120,0,2.5008,0.6184,2.7209,...,0,0,0,0,0,0,0,0,0,0
7,304409,746165,2377318,11.5773,55,55,0,11.5773,0.9060,5.9537,...,0,0,0,0,0,0,0,0,0,0


In [50]:
# summarize the encoded df with the summary statistics
encoded_df.describe()

,ixJobSummary,ixNest,ixPart,dPartTrueArea,cRequired,cNested,fExtShape,dExtArea,dExtBoundaryDist,dExtContainedDist,...,fStrat_Strategy1,fStrat_Strategy10,fStrat_Strategy2,fStrat_Strategy3,fStrat_Strategy4,fStrat_Strategy5,fStrat_Strategy6,fStrat_Strategy7,fStrat_Strategy8,fStrat_Strategy9
count,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,...,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06,4.072515e+06
mean,1.899243e+06,4.511965e+06,1.580208e+07,-7.351136e+286,1.314087e+01,1.060778e+01,5.439213e+00,6.540658e+02,5.553442e+00,3.577339e+01,...,6.790349e-02,6.692940e-02,1.018904e-02,1.868207e-02,2.425803e-01,2.431470e-02,9.286154e-03,9.881854e-03,1.136669e-02,1.475378e-02
std,1.042945e+06,2.441282e+06,8.736020e+06,inf,2.759251e+03,1.338320e+02,1.045352e+01,1.177717e+04,2.978956e+01,5.134109e+01,...,2.515803e-01,2.498997e-01,1.004252e-01,1.353996e-01,4.286434e-01,1.540244e-01,9.591623e-02,9.891514e-02,1.060070e-01,1.205658e-01
min,3.560100e+04,1.007630e+05,3.047800e+05,-3.742202e+292,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.014986e+06,2.468597e+06,8.436690e+06,3.153750e+01,1.000000e+00,1.000000e+00,0.000000e+00,3.266250e+01,1.898800e+00,9.622400e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.905288e+06,4.519524e+06,1.593877e+07,1.058387e+02,2.000000e+00,2.000000e+00,0.000000e+00,1.105994e+02,3.395500e+00,1.900000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.794505e+06,6.605744e+06,2.329271e+07,4.201447e+02,5.000000e+00,5.000000e+00,5.000000e+00,4.382475e+02,6.750000e+00,4.214070e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.688756e+06,8.711716e+06,3.097014e+07,2.960661e+157,5.555555e+06,1.632310e+05,5.400000e+01,1.231630e+07,5.673666e+04,3.239010e+04,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [51]:
# save the result as a .csv that has all the strategies 
# save the encoded df as a .csv file
encoded_df.to_csv('../CEIP_csv/ml_data_withstrats.csv', index=False)

#### Other data cleaning we might do

In [ ]:
# Convert a numeric variable to categorical using custom ranges
# bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
# df['age_group'] = pd.cut(df['age'], bins=bins, labels=lab

#### Downsampling Data
Downsampling data to ~1,000,000 samples for early machine learning & analysis

### Scaling & Normalization
Bring features to a similar scale to prevent one from dominating the  - some ML models sensitive to feature magnitudes, can perform poorly on different scales

In [ ]:
# tree-based models do not require normalization or scaling
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# # Standard Scaling (Z-score normalization)
# scaler = StandardScaler()
# main_df_scaled = pd.DataFrame(scaler.fit_transform(main_df), columns=main_df.columns)

# # Min-Max Scaling (Normalization)
# scaler = MinMaxScaler()
# df['min_max_scaled_feature'] = scaler.fit_transform(df[['column1']])

# Robust Scaling: Scale features using median and interquartile range, making it less sensitive to outliers

**Checking for constant or quasi-constant features:** These are features that have the same value for a large majority of the observations. These features provide no information that allows ML models to predict the target.

In [ ]:
# from sklearn.feature_selection import VarianceThreshold
# constant_filter = VarianceThreshold(threshold=0)
# constant_filter.fit(main_df)
# len(main_df.columns[constant_filter.get_support()])

## Removing outliers

In [ ]:
def remove_outliers_iqr(df, column, threshold=3):
    """
    This function removes the outliers from a dataframe based on the IQR method.
    The function takes as arguments:
        - df: the dataframe
        - column: the column on which to calculate and remove outliers
         - multiplier: the multiplier for the IQR to define the range (default is 3)
    It returns the dataframe with outliers removed.
    """
    # Calculate Q1, Q3, and IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    

    # Define threshold boundaries
    threshold_low = Q1 - 3 * IQR
    threshold_high = Q3 + 3 * IQR

    # Filter out the outliers
    df_filtered = df[(df[column] >= threshold_low) & (df[column] <= threshold_high)]

    return df_filtered

# Apply this function to all numeric columns in the DataFrame
numeric_cols = encoded_df.select_dtypes(include=[np.number]).columns.tolist()

for column in numeric_cols:
    filtered_df = remove_outliers_iqr(encoded_df, column)

In [ ]:
# starting rows: 5,762,62
# ending rows: 4,014,410
# number of rows removed in cleaning: 1,748,212

filtered_df.shape

(4014410, 40)

In [ ]:
# output the results as a cleaned csv 
filtered_df.to_csv('../CEIP_csv/cleaned2.csv', index=False)